In [16]:
#import modules and libraries
import pandas as pd
import yfinance as yf

#data for EUR/USD currency pair from the last 60 days, presented in 15 minutes intervals
dataFrame = yf.download("EURUSD=X", period="60d", interval='30m')
dataFrame.iloc[:,:]

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2023-11-20 00:00:00,1.090750,1.090869,1.090275,1.090275,1.090275,0
2023-11-20 00:30:00,1.090037,1.090631,1.090037,1.090394,1.090394,0
2023-11-20 01:00:00,1.090513,1.091346,1.090513,1.091227,1.091227,0
2023-11-20 01:30:00,1.091227,1.091346,1.090988,1.091227,1.091227,0
2023-11-20 02:00:00,1.091227,1.092419,1.091107,1.092419,1.092419,0
...,...,...,...,...,...,...
2024-02-09 14:30:00,1.078051,1.078981,1.078051,1.078632,1.078632,0
2024-02-09 15:00:00,1.078400,1.079214,1.078400,1.078865,1.078865,0
2024-02-09 15:30:00,1.078981,1.079098,1.078516,1.078865,1.078865,0


### Build a function for the signals:

In [25]:
def getSignal(dataFrame):
    # get prices for open, close, as well as the previous ones
    # to do this, get the last row and the row above it from the table
    lastRows = dataFrame.tail(2)
    open = lastRows['Open'].iloc[0]
    close = lastRows['Close'].iloc[0]
    prevOpen = lastRows['Open'].iloc[1]
    prevClose = lastRows['Close'].iloc[1]
    
    # see if the pattern is bearish, bullish or neither
    if (prevOpen < prevClose
        and open >= prevClose
        and close < prevOpen
        and open > close):
        return 1 # bearish
    elif (prevOpen > prevClose
          and prevOpen < close
          and prevClose >= open
          and open < close):
        return 2 # bullish
    else:
        return 0 # not clear
    
signals = [0]
length = len(dataFrame)
for i in range (1, length):
    signals.append(getSignal(dataFrame[i - 1 : i + 1]))
dataFrame["signal"] = signals
    

In [28]:
dataFrame.iloc[:, :]

,Open,High,Low,Close,Adj Close,Volume,signal
Datetime,,,,,,,
2023-11-20 00:00:00,1.090750,1.090869,1.090275,1.090275,1.090275,0,0
2023-11-20 00:30:00,1.090037,1.090631,1.090037,1.090394,1.090394,0,0
2023-11-20 01:00:00,1.090513,1.091346,1.090513,1.091227,1.091227,0,0
2023-11-20 01:30:00,1.091227,1.091346,1.090988,1.091227,1.091227,0,0
2023-11-20 02:00:00,1.091227,1.092419,1.091107,1.092419,1.092419,0,0
...,...,...,...,...,...,...,...
2024-02-09 14:30:00,1.078051,1.078981,1.078051,1.078632,1.078632,0,0
2024-02-09 15:00:00,1.078400,1.079214,1.078400,1.078865,1.078865,0,0
2024-02-09 15:30:00,1.078981,1.079098,1.078516,1.078865,1.078865,0,0


In [29]:
dataFrame.signal.value_counts()

0    2643
1      72
2      60
Name: signal, dtype: int64

### Connect to market and perform the trades accordingly

In [3]:
from apscheduler.schedulers.blocking import BlockingScheduler
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
from oandapyV20.contrib.requests import MarketOrderRequest
from oanda_candles import Pair, Gran, CandleClient
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails

In [4]:

#parameter - the number of candles that are retrieved
accountID = '62137569'
def getCandles(n):
    access_token='XXXXXXX'
    #you need token here generated from OANDA account
    client = CandleClient(access_token, real=False)
    collector = client.get_collector(Pair.EUR_USD, Gran.M15)
    candles = collector.grab(n)
    return candles

candles = getCandles(3)
for candle in candles:
    print(float(str(candle.bid.o))>1)

HTTPError: 401 Client Error: Unauthorized for url: https://api-fxpractice.oanda.com/v3/instruments/EUR_USD/candles?alignmentTimezone=Etc%2FGMT%2B1&dailyAlignment=23&granularity=M15&price=BAM&weeklyAlignment=Sunday&count=3

In [8]:
def trading_job():
    candles = getCandles(3)
    dfstream = pd.DataFrame(columns=['Open','Close','High','Low'])
    
    i=0
    for candle in candles:
        dfstream.loc[i, ['Open']] = float(str(candle.bid.o))
        dfstream.loc[i, ['Close']] = float(str(candle.bid.c))
        dfstream.loc[i, ['High']] = float(str(candle.bid.h))
        dfstream.loc[i, ['Low']] = float(str(candle.bid.l))
        i=i+1

    dfstream['Open'] = dfstream['Open'].astype(float)
    dfstream['Close'] = dfstream['Close'].astype(float)
    dfstream['High'] = dfstream['High'].astype(float)
    dfstream['Low'] = dfstream['Low'].astype(float)

    signal = signal_generator(dfstream.iloc[:-1,:])#
    
    # EXECUTING ORDERS
    #accountID = "XXXXXXX" #your account ID here
    client = API(access_token)
         
    SLTPRatio = 2.
    previous_candleR = abs(dfstream['High'].iloc[-2]-dfstream['Low'].iloc[-2])
    
    SLBuy = float(str(candle.bid.o))-previous_candleR
    SLSell = float(str(candle.bid.o))+previous_candleR

    TPBuy = float(str(candle.bid.o))+previous_candleR*SLTPRatio
    TPSell = float(str(candle.bid.o))-previous_candleR*SLTPRatio
    
    print(dfstream.iloc[:-1,:])
    print(TPBuy, "  ", SLBuy, "  ", TPSell, "  ", SLSell)
    signal = 2
    #Sell
    if signal == 1:
        mo = MarketOrderRequest(instrument="EUR_USD", units=-1000, takeProfitOnFill=TakeProfitDetails(price=TPSell).data, stopLossOnFill=StopLossDetails(price=SLSell).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)
    #Buy
    elif signal == 2:
        mo = MarketOrderRequest(instrument="EUR_USD", units=1000, takeProfitOnFill=TakeProfitDetails(price=TPBuy).data, stopLossOnFill=StopLossDetails(price=SLBuy).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)

In [9]:
trading_job()

#scheduler = BlockingScheduler()
#scheduler.add_job(trading_job, 'cron', day_of_week='mon-fri', hour='00-23', minute='1,16,31,46', start_date='2022-01-12 12:00:00', timezone='America/Chicago')
#scheduler.start()

HTTPError: 401 Client Error: Unauthorized for url: https://api-fxpractice.oanda.com/v3/instruments/EUR_USD/candles?alignmentTimezone=Etc%2FGMT%2B1&dailyAlignment=23&granularity=M15&price=BAM&weeklyAlignment=Sunday&count=3